<img src = "https://github.com/Trading-com-Dados/pmf/blob/main/logotipo-trading-com-dados.svg?raw=true" width = "300">

---

# **Obtenção e análise de dados de FIIs e criação de dashboard para tomada de decisão de investimento**

In [1]:
%pip install selenium
%pip install vizro
%pip install yfinance

   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.5 MB 5.7 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.5 MB 30.9 MB/s eta 0:00:01
   ------------- -------------------------- 3.1/9.5 MB 32.8 MB/s eta 0:00:01
   ------------- -------------------------- 3.2/9.5 MB 16.8 MB/s eta 0:00:01
   -------------- ------------------------- 3.4/9.5 MB 14.7 MB/s eta 0:00:01
   ---------------- ----------------------- 3.9/9.5 MB 13.7 MB/s eta 0:00:01
   -------------------- ------------------- 4.9/9.5 MB 15.7 MB/s eta 0:00:01
   --------------------------- ------------ 6.4/9.5 MB 17.9 MB/s eta 0:00:01
   --------------------------------- ------ 7.8/9.5 MB 19.3 MB/s eta 0:00:01
   --------------------------------- ------ 7.9/9.5 MB 17.3 MB/s eta 0:00:01
   --------------------------------- ------ 8.0/9.5 MB 15.5 MB/s eta 0:00:01
   ----------------------------------- ---- 8.3/9.5 MB 15.2 MB/s eta 0:00:01
   -----

In [2]:
import pandas as pd


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import matplotlib.pyplot as plt
import vizro.plotly.express as px
from vizro import Vizro
import vizro.models as vm

import yfinance as yf

# **Iniciar o crawling**

In [3]:
driver = webdriver.Chrome()

Vamos utilizar este site: https://www.fundsexplorer.com.br/ranking

In [4]:
driver.get('https://www.fundsexplorer.com.br/ranking')

# **Extração da tabela original**

In [5]:
driver.find_element(By.XPATH,'//*[@id="upTo--default-fiis-table"]/div/table')

<selenium.webdriver.remote.webelement.WebElement (session="69b1dbfbfc3bc89975c5bf99a9f060e3", element="f.71F50647799CDC562A9BEFCBBDE2893B.d.3E97BC807705AB34A4DE5671CA7D94A1.e.614")>

In [6]:
tb_din = driver.find_element(By.XPATH,'//*[@id="upTo--default-fiis-table"]/div/table').get_attribute('outerHTML')

In [7]:
fiis = pd.read_html(tb_din, decimal = ',', thousands = '.')[0]

C:\Users\dri_f\AppData\Local\Temp\ipykernel_13728\75911920.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  fiis = pd.read_html(tb_din, decimal = ',', thousands = '.')[0]


**Manipulação dos dados**

In [8]:
fiis.shape

(463, 8)

In [9]:
fiis.head()

,Fundos,Setor,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido,Num. Cotistas
0,AAZQ11,Indefinido,862219.09,0.90,"1,25 %","-4,13 %",2.308578e+08,27598
1,ABCP11,Shoppings,51970.96,0.68,"0,67 %","-2,16 %",4.775612e+08,16563
2,AFHI11,Papéis,1344066.23,1.01,"0,97 %","-0,33 %",3.828339e+08,34676
3,AGRX11,Outros,289777.43,NaN,"1,35 %","-7,57 %",NaN,0
4,AIEC11,Lajes Corporativas,363313.52,0.64,"1,34 %","0,28 %",4.066412e+08,17213


In [10]:
fiis = fiis.replace({',':'.', '%':''}, regex = True)

In [11]:
fiis[['Dividend Yield', 'Variação Preço']] = fiis[['Dividend Yield', 'Variação Preço']].apply(pd.to_numeric)

In [12]:
fiis.dtypes

Fundos                   object
Setor                    object
Liquidez Diária (R$)    float64
P/VP                    float64
Dividend Yield          float64
Variação Preço          float64
Patrimônio Líquido      float64
Num. Cotistas             int64
dtype: object

In [13]:
fiis = fiis.set_index('Fundos')

In [14]:
fiis

,Setor,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido,Num. Cotistas
Fundos,,,,,,,
AAZQ11,Indefinido,862219.09,0.90,1.25,-4.13,2.308578e+08,27598
ABCP11,Shoppings,51970.96,0.68,0.67,-2.16,4.775612e+08,16563
AFHI11,Papéis,1344066.23,1.01,0.97,-0.33,3.828339e+08,34676
AGRX11,Outros,289777.43,NaN,1.35,-7.57,NaN,0
AIEC11,Lajes Corporativas,363313.52,0.64,1.34,0.28,4.066412e+08,17213
...,...,...,...,...,...,...,...
YUFI11,Fundo de Desenvolvimento,90.00,0.85,0.00,0.00,5.189643e+07,59
YUFI11B,Indefinido,NaN,NaN,0.00,0.00,NaN,0
ZAGH11,Serviços Financeiros Diversos,1785.00,1.04,0.00,0.00,4.685290e+07,73


In [15]:
fiis.fillna(0, inplace = True)

In [16]:
fiis['FII'] = fiis.index
fiis['Setor'] = fiis['Setor'].astype(str)

In [17]:
fiis.sort_values('P/VP', ascending = False)

,Setor,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido,Num. Cotistas,FII
Fundos,,,,,,,,
PRTS11,Shoppings,0.00,1961.67,0.00,0.00,3111607.44,11,PRTS11
SRVD11,Indefinido,233.91,583.00,2.29,-9.79,-991836.94,106,SRVD11
IBFF11,Fundo de Fundos,162549.50,4.49,0.00,0.00,9855905.16,3870,IBFF11
RBRM11,Fundo de Desenvolvimento,0.00,3.76,0.00,0.00,13579348.87,54,RBRM11
LOFT11B,Fundo de Desenvolvimento,0.00,3.02,0.00,0.00,19433901.34,1,LOFT11B
...,...,...,...,...,...,...,...,...
RBCO11,Lajes Corporativas,0.00,0.00,0.00,0.00,0.00,0,RBCO11
ESUD11,Indefinido,0.00,0.00,0.00,0.00,0.00,0,ESUD11
EQIA11,Fiagro,0.00,0.00,0.00,0.00,0.00,0,EQIA11


# **Criação do Dashboard de FIIs**

In [18]:
page = vm.Page(
    title = 'Dashboard de FIIs',
    components = [
        vm.Graph(id = 'scatter_dy', figure = px.scatter(fiis, x ='Num. Cotistas' , y ='Dividend Yield', color = 'Setor')),
        vm.Graph(id = 'bar_pvp', figure = px.bar(fiis, x ='FII' , y ='P/VP', color = 'Setor')),
        vm.Graph(id = 'bar_patliq', figure = px.bar(fiis, x ='FII' , y ='Patrimônio Líquido', color = 'Setor')),
        
        
    ],
    
    controls = [
        vm.Filter(column = 'Setor'),
        vm.Filter(column = 'Liquidez Diária (R$)', selector = vm.RangeSlider()),
        vm.Filter(column = 'Dividend Yield', selector = vm.RangeSlider()),
        vm.Filter(column = 'Num. Cotistas', selector = vm.RangeSlider())
        
        
    ]
    

)

dashboard = vm.Dashboard(pages = [page])

Vizro().build(dashboard).run()